# Disseration Experiment 4k
# Generate SHAP XAI Output (Credit Card Fraud) - Feb 18¶
Ciaran Finnegan February 2024

# Import Libraries + Custom Functions

## Import Libraries

In [ ]:
# Import libs
import numpy as np
import pandas as pd

# Import Display libraries
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns
from prettytable import PrettyTable
import raiutils
from raiutils.exceptions import UserConfigValidationException


# Import necessary libraries for ANN model building
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Import necessary libraries for LIME calculations
import lime
import lime.lime_tabular

# Libraries required for metrics calculations
from scipy.spatial import distance
from sklearn.cluster import KMeans

from sklearn.preprocessing import LabelEncoder
import warnings
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample


# Classifier training (not used for explainability)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Libraries used in Experiment Creation of XL Output Metrics
import os
import time
import random
import openpyxl

## Custom Functions

Dataset Visualisations

In [ ]:
%run ./DS_Visualisation_Functions.ipynb

Metrics

In [ ]:
%run ./XAI_Metrics_Functions.ipynb

Model Evaluation Functions

In [ ]:
%run ./DS_Model_Build_Evaluation_Functions.ipynb

Track Experiment Result Functions

In [ ]:
%run ./XAI_Experiment_Functions.ipynb

# Load Model

A Neural Network Model has been created in another Kubeflow Notebook and is being used in all the XAI experiments

In [ ]:
loaded_model = keras.models.load_model('ccfraud_model')  # If saved as SavedModel

In [ ]:
X_test_loaded, y_test_loaded, X_train_loaded, y_train_loaded, df_downsampled_loaded, dfCatCols = load_CC_train_test_data()

In [ ]:
#X_train_loaded.head(1)

In [ ]:
#y_train_loaded.head(2)

## Re-Display Model Peformance

For illustration, the evualtion metrics of the NN model will be repeated here.

### Re-Scale Data

In [ ]:
X_train_loaded_scaled, X_test_loaded_scaled, scale_loaded = scale_the_features(X_train_loaded, 
                                                                                X_test_loaded, 
                                                                                df_downsampled_loaded, 
                                                                                'Fraud')

In [ ]:
# Set option to display all columns (you can adjust the number as needed)
pd.set_option('display.max_columns', None)

In [ ]:
#X_train_loaded_scaled

In [ ]:
#X_train_loaded_scaled

### Re-evaluate loaded model

In [ ]:
y_pred_loaded = display_model_metrics_tabular(loaded_model, X_test_loaded_scaled, y_test_loaded)

### Confusion Matrix

In [ ]:
generate_confusion_matrix(y_test_loaded, y_pred_loaded)

# Generate LIME Values (Examples Instances)

Set Up Column Names for Scaling

In [ ]:
# Assuming 'X_test' is a numpy array and you have a list of the original column names
# original_feature_names = [col for col in df_downsampled_loaded.columns if col != 'Fraud']

## Select Value + Prepare Function

In [ ]:
# Importing necessary libraries - required here to ensure'lime_tabular' routines are available
from lime import lime_tabular

In [ ]:
#Jan 6th - use new loaded model
column_names = df_downsampled_loaded.drop('Fraud', axis=1).columns

In [ ]:
# Convert NumPy array to DataFrame
X_test_loaded_scaled = pd.DataFrame(X_test_loaded_scaled, columns=column_names)

In [ ]:
#Jan 18th - use new loaded mode, and SCALED data
random_observation = X_test_loaded_scaled.sample(1, random_state=42)

In [ ]:
instance = random_observation.values.reshape(1, -1)

In [ ]:
# Wrapper function to output probabilities for both classes
def predict_proba_wrapper(data):
    
    # Get the probability of the positive class
    prob_pos = loaded_model.predict(data)
    
    # Get the probability of the negative class
    prob_neg = 1 - prob_pos
    
    # Combine and return the probabilities
    return np.hstack([prob_neg, prob_pos])

In [ ]:
# Convert NumPy array to DataFrame
X_train_loaded_scaled = pd.DataFrame(X_train_loaded_scaled, columns=column_names)

## Single Instance LIME Explanation

In [ ]:
# Jan 18th - use new loaded model and data - SCALED
explainer = lime_tabular.LimeTabularExplainer(X_train_loaded_scaled.values, 
                                              mode='classification',
                                              training_labels=y_train_loaded,
                                              class_names=['Non Fraud', 'Fraud'], 
                                              feature_names=column_names)

In [ ]:
# 6. Explaining the prediction using the LIME explainer
# Using our wrapper function for predictions
explanation = explainer.explain_instance(instance[0], predict_proba_wrapper)

In [ ]:
# Get the prediction for the instance
prediction_probs1 = predict_proba_wrapper(instance[0].reshape(1, -1))

In [ ]:
print(f'Predicted Probability (Fraud): {prediction_probs1[0][1]:.2f}\n')

In [ ]:
# Displaying the LIME explanations for the instance
# 'show_in_notebook' displays the explanation in a Jupyter notebook format
explanation.show_in_notebook(show_table=True)

## Build LIME Explainers for XAI Metrics

### Set Up Explainer for LIME Metrics

In [ ]:
# Create a LIME explainer object with a new name
# Jan 18th - use new loaded model and data - SCALED
lime_explainer = lime.lime_tabular.LimeTabularExplainer(X_train_loaded_scaled.values, 
                                                        feature_names=X_train_loaded.columns.tolist(), 
                                                        class_names=['Non Fraud', 'Fraud'], 
                                                        mode='classification')

In [ ]:
# Adjusting the prediction function to ensure it provides probabilities for both classes
# Jan 6th - use new loaded model and data
def predict_fn(data):
    prob_pos = loaded_model.predict(data)
    prob_neg = 1 - prob_pos
    return np.hstack([prob_neg, prob_pos])

### Sample Feature LIME Values

In [ ]:
def detailed_explanation(instance, real_value):
    """
    Generate LIME explanation for a specific instance and provide a detailed narrative.
    
    Parameters:
    - instance (numpy array): The instance for which the LIME explanation is to be generated.
    - real_value (int): The true label of the instance.
    
    Returns:
    - exp (Lime explanation object): The LIME explanation for the instance.
    """

    # Get the prediction for the instance
    prediction_probs = predict_fn(instance.reshape(1, -1))
    
    # Generate LIME explanation for the instance using the modified prediction function  
    # Jan 6th - use new loaded model and data
    #exp = lime_explainer.explain_instance(instance, predict_fn, num_features=len(X_train_loaded.columns))
    exp = lime_explainer.explain_instance(instance, predict_fn, num_features=15)
    
    # Display the LIME explanation in a readable format
    print(f'\n\n----------------------------------------')
    print(f'Real Value: {real_value}')
    print(f'Predicted Probability (Fraud): {prediction_probs[0][1]:.2f}\n')
    print('Features in Order of Importance:')
    for feature, weight in sorted(exp.as_list(), key=lambda x: abs(x[1]), reverse=True):
        feature_parts = feature.split(' ')
        feature_name = feature_parts[0]
        feature_value = ' '.join(feature_parts[1:])
        print(f'Feature: {feature_name} | Value: {feature_value} | Weight: {weight:.2f}')
    print('----------------------------------------\n\n')
    
    return exp

# The function is now ready to be used. When called, it will provide a detailed explanation for a given instance.

In [ ]:
def detailed_explanation_reverseScaled(instance, real_value, scale_loaded, feature_names):
    """
    Generate LIME explanation for a specific instance and provide a detailed narrative.
    
    Parameters:
    - instance (numpy array): The instance for which the LIME explanation is to be generated.
    - real_value (int): The true label of the instance.
    - scale_loaded (StandardScaler object): The scaler used for inverse transforming the features.
    
    Returns:
    - exp (Lime explanation object): The LIME explanation for the instance.
    """
    
    # Convert feature_names to a list if it's a Pandas Index
    if isinstance(feature_names, pd.Index):
        feature_names = feature_names.tolist()
        
        

    # Get the prediction for the instance
    prediction_probs = predict_fn(instance.reshape(1, -1))
    
    # Generate LIME explanation for the instance using the modified prediction function
    exp = lime_explainer.explain_instance(instance, predict_fn, num_features=7)
    
    # Inverse transform the scaled instance
    original_instance = scale_loaded.inverse_transform(instance.reshape(1, -1)).flatten()
    
    # Display the LIME explanation in a readable format
    print(f'\n\n----------------------------------------')
    print(f'Real Value: {real_value}')
    print(f'Predicted Probability (Fraud): {prediction_probs[0][1]:.2f}\n')
    print('Features in Order of Importance:')
    for feature, weight in sorted(exp.as_list(), key=lambda x: abs(x[1]), reverse=True):
        feature_parts = feature.split(' ')
        feature_name = feature_parts[0]
        # Check if feature_name is non-numeric before using it to look up the index
        if is_non_numeric(feature_name):
            feature_index = feature_names.index(feature_name)
            original_feature_value = original_instance[feature_index]
            print(f'Feature: {feature_name} | Value: {original_feature_value} | Weight: {weight:.2f}')
        else:
            print(f'Feature: {feature} | Weight: {weight:.2f}')
    print('----------------------------------------\n\n')
    
    
    return exp

In [ ]:
# Generating LIME explanations for the first three values in the test dataset
# Jan 18th - use new loaded model and data - SCALED
for i in range(2):
    instance = X_test_loaded_scaled.iloc[i].values
    real_value = y_test_loaded.iloc[i]
    detailed_explanation(instance, real_value)

In [ ]:
#column_names

In [ ]:
def is_non_numeric(s):
    """
    Check if a string is non-numeric (contains alphanumeric characters).

    Parameters:
    - s (str): The string to be checked.

    Returns:
    - bool: True if the string is non-numeric, False otherwise.
    """
    return not s.replace('.', '', 1).isdigit()

In [ ]:
# Generating LIME explanations for the first three values in the test dataset
# Jan 18th - use new loaded model and data - SCALED
for i in range(2):
    instance = X_test_loaded_scaled.iloc[i].values
    real_value = y_test_loaded.iloc[i]
    #detailed_explanation_reverseScaled(instance, real_value, scale_loaded, column_names)

Narrative:

For each instance, the output will display the features in order of their importance in determining the predicted probability.

For the instance with a real fraud value of 1:

As the predicted probability of a fraudulent transaction is high, this means the model recognizes patterns in this instance similar to other fraudulent instances from the training data.

Features with positive weights contributed to increasing the probability of a fraud prediction. The larger the weight, the more influential that feature was.

Conversely, features with negative weights worked against the fraud prediction. The larger the negative weight, the more it tried to reduce the probability.




For the two instances with a real fraud value of 0:

If the predicted probability of a fraudulent transaction is low, this means the model recognizes patterns in this instance similar to other non-fraud instances from the training data.

Again, positive weights indicate features that tried to push the prediction towards fraud, while negative weights indicate features that tried to push the prediction away from fraud.




For each feature displayed, the "Value" tells you what the specific value of that feature was for the instance, and the "Weight" tells you how much that feature influenced the prediction. By examining the top features and their weights, you can get a good understanding of why the model made its prediction.







### Prediction  Outputs

In [ ]:
#X_test_loaded_scaled
# 1. Direct Evaluation
# Jan 18th - use new loaded model and data - SCALED
sample_instance = X_test_loaded_scaled.iloc[0].values.reshape(1, -1)
print("Direct model prediction:", loaded_model.predict(sample_instance))
print("predict_fn output:", predict_fn(sample_instance))

In [ ]:
# 2. Check Output Shape
# Jan 6th - use new loaded model and data
assert predict_fn(sample_instance).shape == (1, 2), "Output shape of predict_fn is incorrect!"

In [ ]:
# 3. Check Probability Sum
# Jan 6th - use new loaded model and data
probabilities = predict_fn(sample_instance)
assert abs(probabilities[0].sum() - 1) < 1e-6, "Probabilities do not sum to 1!"

In [ ]:
#X_test_loaded_scaled
# 4. Visual Inspection
# Jan 18th - use new loaded model and data - SCALED
print("\nVisual inspection of predict_fn outputs for first 5 test instances:")
for i in range(5):
    instance = X_test_loaded_scaled.iloc[i].values.reshape(1, -1)
    print(f"Instance {i}: {predict_fn(instance)}")

# Prepare LIME Values for Metrics

To achieve your goal, we need to:

Generate explanations for the chosen instances.
Create dataframes for the individual instance features and their corresponding LIME values/weights.
Ensure consistent structure for the LIME values/weights dataframe.
Save the dataframes to output files.
Given the specific structure you need for the LIME values/weights dataframe, we'll also have to ensure that each feature has a consistent number of columns across all instances.

Let's proceed:

We first choose the desired instances, two with a target value of 1 and four with a target value of 0.

The get_detailed_explanation function generates LIME explanations for an instance 
and returns the feature values and LIME values/weights.

## Generate LIME Details 

In [ ]:
# Function to get LIME explanations for an instance, function returns explanations
def get_detailed_explanation(instance):
    """
    Generate LIME explanation for a specific instance.
    
    Parameters:
    - instance (numpy array): The instance for which the LIME explanation is to be generated.
    
    Returns:
    - feature_values (list): List of feature values for the instance.
    - lime_values (list): List of tuples containing LIME values and weights for the instance.
    """
    # Generate LIME explanation for the instance
    print("Instance type: ", type(instance))
    print("Instance length: ", len(instance))
    
    exp = lime_explainer.explain_instance(instance,
                                          predict_fn,
                                          num_features=32)
                                          #num_features=len(X_train_loaded.columns))
    
    feature_values = instance.tolist()
    lime_values = exp.as_list()
    
    return feature_values, lime_values

We then loop over the chosen instances, get their feature values and 
LIME explanations, and store them in lists.

## Generate LIME Explanations

In [ ]:
@timeit
def generate_lime_explanations(df, chosen_indices):
    
    # Lists to store instance features and LIME values/weights
    features_data = []
    lime_data = []

    # Generate explanations for the chosen instances
    for index in chosen_indices:
        print(f'Index is {index}')
        feature_values, lime_values = get_detailed_explanation(df.loc[index].values)
        features_data.append(feature_values)
        lime_data.append(lime_values)
    
    # Jan 6th - use new loaded model and data
    # Convert features data to a DataFrame
    features_df = pd.DataFrame(features_data, columns=X_train_loaded.columns.tolist())

    # Ensure consistent structure for the LIME values/weights dataframe
    max_columns = max(len(lime_values) for lime_values in lime_data) * 2  # Multiply by 2 to account for values and weights
    lime_array = []

    for lime_values in lime_data:
        lime_values_array = []
        for feature, weight in lime_values:
            lime_values_array.extend([feature, weight])
        # Padding to ensure consistent number of columns
        lime_values_array.extend([None] * (max_columns - len(lime_values_array)))
        lime_array.append(lime_values_array)

    lime_df = pd.DataFrame(lime_array)
    
    return features_df, lime_df

In [ ]:
import re

def extract_feature_names2(feature_descriptors):
    """
    Extracts feature names from a list of LIME explanation descriptors, 
    accommodating complex feature names including periods and other special characters.

    Parameters:
    - feature_descriptors (list of str): A list of strings containing LIME explanation descriptors.

    Returns:
    - list of str: A list containing the extracted feature names.
    """
    
    # Define the regex pattern to match feature names, allowing for periods, underscores, and alphanumeric characters
    pattern = r'(?<![\d<>=])\b[a-zA-Z_.]+[a-zA-Z0-9_.]*\b'
    
    # Initialize an empty list to store the extracted feature names
    raw_feature_names = []
    
    # Iterate over each descriptor in the input list
    for descriptor in feature_descriptors:
        # Use regex to find all substrings that match the pattern of a feature name
        matches = re.findall(pattern, descriptor)
        if matches:
            # Assuming the first match is the feature name, add it to the list
            raw_feature_names.append(matches[0])
        else:
            # Fallback in case no match is found (shouldn't happen with correct input)
            raw_feature_names.append(descriptor)
    
    # Print the extracted feature names
    print("Extracted Feature Names:")
    #for name in raw_feature_names:
    #    print(name)
    
    # Return the list of extracted feature names
    return raw_feature_names

In [ ]:
# Example usage
feature_descriptors = [
    'CustomerPresentIndicator_N > 1.12', 
    '-1.16 < TxnChannelCode_OnL <= 0.86', 
    '>= 2.5 Age', 
    'Income <= -0.5',
    '0.5 < DomesticAuthCount.cnt.hour1 <= 1.0'
]

In [ ]:
# Call the function and get the list of feature names
extracted_feature_names = extract_feature_names2(feature_descriptors)

## Generate LIME Explanations 2

In [ ]:
def extract_feature_names(feature_descriptors):
    """
    Extracts feature names from a list of LIME explanation descriptors.

    Parameters:
    - feature_descriptors (list of str): A list of strings containing LIME explanation descriptors.

    Returns:
    - list of str: A list containing the extracted feature names.
    """
    pattern = r'(?<![\d<>=])\b[a-zA-Z_.]+[a-zA-Z0-9_.]*\b'
    raw_feature_names = []
    for descriptor in feature_descriptors:
        matches = re.findall(pattern, descriptor)
        if matches:
            raw_feature_names.append(matches[0])
        else:
            raw_feature_names.append(descriptor)
    
    # Optionally, print the names
    #print("Extracting Feature Names:")
    #for name in raw_feature_names:
    #    print(name)
    #print("Feature Names Extracted:")
    
    return raw_feature_names

In [ ]:
def extract_feature_names_with_default(feature_descriptors, all_feature_names):
    """
    Extracts feature names and aligns them with the original feature names, setting default weight to 0.

    Parameters:
    - feature_descriptors (list of tuples): LIME explanation descriptors and weights.
    - all_feature_names (list of str): All feature names in the order they appear in the instance.

    Returns:
    - list of tuples: A list containing the feature names and weights, aligned with all_feature_names.
    """
    # Create a dictionary for quick lookup of weights by feature name
    feature_weights = {re.findall(r'(?<![\d<>=])\b[a-zA-Z_.]+[a-zA-Z0-9_.]*\b', desc)[0]: weight for desc, weight in feature_descriptors}
    
    # Create structured list with default weight of 0 for features not in explanations
    structured_list = [(feature, feature_weights.get(feature, 0)) for feature in all_feature_names]
    
    return structured_list

In [ ]:
def get_detailed_explanation2(instance):
    """
    Generate LIME explanation for a specific instance and extract feature names.

    Parameters:
    - instance (numpy array): The instance for which the LIME explanation is to be generated.
    
    Returns:
    - feature_values (list): List of feature values for the instance.
    - lime_values (list): List of tuples containing LIME values and weights for the instance.
    - feature_names (list): List of feature names for which LIME provided explanations.
    """

    exp = lime_explainer.explain_instance(instance,
                                          predict_fn,
                                          num_features=16)
                                          
    feature_values = instance.tolist()
    lime_values = exp.as_list()
    
    # Extract the feature names from the LIME explanations
    feature_descriptors = [item[0] for item in lime_values]
    feature_names = extract_feature_names(feature_descriptors)
    
    
    # Get all feature names in the order they appear in the instance
    all_feature_names = X_train_loaded.columns.tolist()
    
    # Extract the feature names and weights, and align them with all features, defaulting to 0
    lime_values_structured = extract_feature_names_with_default(lime_values, all_feature_names)
    
    
    return feature_values, lime_values, feature_names, lime_values_structured

In [ ]:
@timeit
def generate_lime_explanations2(df, chosen_indices):
    
    # Lists to store instance features and LIME values/weights
    features_data = []
    lime_data = []

    # Generate explanations for the chosen instances
    for index in chosen_indices:
        print(f'Index is {index}')
        feature_values, lime_values, feature_names, lime_values_structured = get_detailed_explanation2(df.loc[index].values)
        features_data.append(feature_values)
        lime_data.append(lime_values)
    
    # Jan 6th - use new loaded model and data
    # Convert features data to a DataFrame
    features_df = pd.DataFrame(features_data, columns=X_train_loaded.columns.tolist())

    # Ensure consistent structure for the LIME values/weights dataframe
    max_columns = max(len(lime_values) for lime_values in lime_data) * 2  # Multiply by 2 to account for values and weights
    lime_array = []

    for lime_values in lime_data:
        lime_values_array = []
        for feature, weight in lime_values:
            lime_values_array.extend([feature, weight])
        # Padding to ensure consistent number of columns
        lime_values_array.extend([None] * (max_columns - len(lime_values_array)))
        lime_array.append(lime_values_array)

    lime_df = pd.DataFrame(lime_array)
    
    return features_df, lime_df, lime_values_structured

In [ ]:
@timeit
def generate_lime_explanations3(df, chosen_indices):
    # Lists to store instance features, LIME values/weights, and structured LIME explanations
    features_data = []
    lime_data = []
    lime_structured_data = []  # New list to store structured LIME data

    # Generate explanations for the chosen instances
    for index in chosen_indices:
        print(f'Index is {index}')
        feature_values, lime_values, feature_names, lime_values_structured = get_detailed_explanation2(df.loc[index].values)
        features_data.append(feature_values)
        lime_data.append(lime_values)
        lime_structured_data.append(lime_values_structured)  # Append structured LIME explanations

    # Convert features data to a DataFrame - function output 1
    features_df = pd.DataFrame(features_data, columns=X_train_loaded.columns.tolist())
    
    
    
    # Ensure consistent structure for the LIME values/weights dataframe
    max_columns = max(len(lime_values) for lime_values in lime_data) * 2  # Multiply by 2 to account for values and weights
    lime_array = []

    for lime_values in lime_data:
        lime_values_array = []
        for feature, weight in lime_values:
            lime_values_array.extend([feature, weight])
        # Padding to ensure consistent number of columns
        lime_values_array.extend([None] * (max_columns - len(lime_values_array)))
        lime_array.append(lime_values_array)

    # Convert feature Lime explanation and weights data to a DataFrame - - function output 2    
    lime_df = pd.DataFrame(lime_array)

    
    
    # Create a DataFrame for structured LIME data that alternates feature names and LIME weights
    # Prepare column names for the new DataFrame
    feature_names = X_train_loaded.columns.tolist()
    column_names = []
    for i, feature_name in enumerate(feature_names, start=1):
        column_names.append(feature_name)  # Feature name
        column_names.append(f'Feature_Weight_{i}')  # Corresponding weight column

    # Convert structured LIME data into a format suitable for DataFrame construction
    lime_structured_array = []
    for lime_values_structured in lime_structured_data:
        row_data = []
        for feature, weight in lime_values_structured:
            row_data.extend([feature, weight])
        lime_structured_array.append(row_data)

    # Create the DataFrame with structured LIME explanations - function output 3
    lime_structured_df = pd.DataFrame(lime_structured_array, columns=column_names)
    
    
    
    return features_df, lime_df, lime_structured_df

The provided Python function; 

  --  get_detailed_explanation(instance)  --

is designed to generate explanations for a specific instance in the credit card fraud dataset using the LIME (Local Interpretable Model-agnostic Explanations) framework. 

Understanding... 

  -- exp.as_list() --


This method call returns a list of tuples, where each tuple contains two elements:

 - Feature Name or Descriptor: A string describing a feature (input variable) of the instance being explained. For a credit card fraud dataset, this could be features like transaction amount, transaction time, or various anonymized features often found in such datasets.
 
 - LIME Value (Weight or Coefficient): A floating-point number indicating the weight or importance of the feature towards the model's prediction for this specific instance. This value helps to understand how each feature contributes to the model's prediction, whether positively (increasing the likelihood of being classified as fraud) or negatively (decreasing the likelihood of being classified as fraud).
 
The output from exp.as_list() essentially provides a local explanation for why the model made a specific prediction for an instance. Each tuple in the list indicates the direction and magnitude of a feature's impact on the prediction, giving insights into the model's behavior at a fine-grained level.

The feature values are straightforwardly converted to a dataframe.
For the LIME values/weights dataframe, we ensure a consistent number of columns 
across all instances by determining the maximum number of columns needed (max_columns) 
and then padding each instance's LIME values/weights with None values to match this maximum.

# XAI Experiments - Metrics Capture

## Suppress Warnings to clean up output

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

## Break out Model Test Data into a list of dataframes

### Create Test Data for Experiment Input

Ensure 'X_test' and 'y_test' Are DataFrames with Proper Columns

In [ ]:
# Assuming 'X_test' is a numpy array and you have a list of the original column names
original_feature_names = [col for col in df_downsampled_loaded.columns if col != 'Fraud']

# Ensure X_test_loaded has the correct column names (if necessary)
X_test_loaded.columns = original_feature_names

In [ ]:
# Combine X_test_loaded and y_test into a single DataFrame
df_TestData = pd.concat([X_test_loaded, y_test_loaded], axis=1)

### Split the DataFrame into 20 consecutive smaller DataFrames

In [ ]:
# Split the DataFrame into 20 consecutive smaller DataFrames
split_size, list_df = split_TestData_into_nn_Blocks(df_TestData, num_splits = 20)

### Check Label Count for Stability Metrics

In [ ]:
# Count the occurrence of each unique value in the 'Fraud' column
fraud_counts = df_TestData['Fraud'].value_counts()

# Display the counts
print("Breakdown of 'Fraud' and non-Fraud label records in df_TestData:")
print(fraud_counts)

### Add a routine to check output values

In [ ]:
# Display starting points in the first nn sub dataframes
startBlockDisplay(df_TestData, split_size, 1)

## Confirm Starting Point in External LIME XAI XL File

The code below acts so that for each dataframe in the list just created the following actions are carried out;

Check if an XAI results XL spreadsheet called 'LIME_XAI_Metrics_Experiments.xls' exists;

If not create an empty XL spreadsheet with the name 'LIME_XAI_Metrics_Experiments.xls', and then define a variable called ‘Sample’ with an integer value of 1 and print the value of 'Sample' to output.

If and XL spreadsheet called 'LIME_XAI_Metrics_Experiments.xls' does exist, then read the entries in the spreadsheet in the first column named ‘Sample Number’ and create a variable in this Python program named ‘Sample’ that is one integer value higher than the highest integer number column named ‘Sample Number’ in the XL, and print this value of 'Sample' to output.

In [ ]:
# Create a sequential number as an identifier for each DataFrame
list_df = {f'df_{i + 1}': list_df[i] for i in range(len(list_df))}

In [ ]:
# File path for the LIME XAI metrics results spreadsheet
LIME_xai_file_path = 'LIME_XAI_Metrics_Experiments.xlsx'  # Stored locally

In [ ]:
# Call Function to update or create the spreadsheet and determine the 'Sample' number
# Process each dataframe in 'list_df'
sample = return_next_sample_number_to_process(list_df, LIME_xai_file_path, "LIME")

## Select Next Dataframe to Process

---------------------------------

	
Extend the Python code so that the code reads in the dataframe from 'list df' that corresponds to the integer value in the 
variable named ‘Sample’. 

Assign this dataframe the name 'df_Selected_from_List'.


----------------------------------

### Initialize Dataframe to Capture Re-start Point as None

In [ ]:
# Initialize df_Selected_from_List as None
df_Selected_from_List = None

### Extract test data block to restart XAI metrics process

In [ ]:
df_Selected_from_List, key = select_restart_testdata_block(df_Selected_from_List, 
                                                           list_df, 
                                                           LIME_xai_file_path)

In [ ]:
# If no DataFrame is selected (e.g., if 'Sample' exceeds the number of DataFrames in list_df)
if 'df_Selected_from_List' not in locals():
    print("No DataFrame selected. The 'Sample' number may exceed the number of DataFrames in list_df.")

-----------------------------

## Generate XAI Metrics from Dataframe

### Generate the LIME Values for the Test Data Block

#### Pre-Process Values for Data Block

In [ ]:
#df_Selected_from_List.head(2)

In [ ]:
#print(df_Selected_from_List.index)

#### Scale the feature values

Call Scaling Function

In [ ]:
#original_feature_names
# Assuming 'X_test' is a numpy array and you have a list of the original column names
original_feature_names = [col for col in df_downsampled_loaded.columns if col != 'Fraud']

In [ ]:
# Scale the feature inputs so that they work with the SHAP generation processs
df_Selected_Scaled_Data_from_List = scale_feature_inputs(df_Selected_from_List, 
                                                         original_feature_names)

In [ ]:
#df_Selected_Scaled_Data_from_List

In [ ]:
#print(df_Selected_Scaled_Data_from_List.index)

In [ ]:
# Dynamically determine start and end indices
start_index = df_Selected_from_List.index[0]  # First index of the DataFrame
print(f'Start Index is : {start_index}')

In [ ]:
end_index = df_Selected_from_List.index[-1]  # Last index of the DataFrame
print(f'End Index is : {end_index}')

In [ ]:
# Get the range of indices as a list
index_range = list(range(df_Selected_Scaled_Data_from_List.index[0],
                         df_Selected_Scaled_Data_from_List.index[-1] + 1))

In [ ]:
#print(index_range)

Set limit value (for debugging)

In [ ]:
# A 'zero' limit value will process the entire data block
limit_data_block_rows = 0 # A zero value processes the entire data block

In [ ]:
# Jan 6th - use new loaded model and data
default_1_indices = index_range

In [ ]:
# For debugging allow a partial subset of data in the data block to be processes
if limit_data_block_rows > 0:
    chosen_indices = list(np.random.choice(default_1_indices, limit_data_block_rows)) 
else:
    chosen_indices = [i for i in range(65)]   

In [ ]:
#print(chosen_indices)

In [ ]:
chosen_indices.sort()

In [ ]:
#print(chosen_indices)

Extract the label values from the data block

In [ ]:
y_test_block_labels_df = df_Selected_Scaled_Data_from_List['Fraud']

In [ ]:
#y_test_block_labels_df.shape

#### Get LIME Values for Data Block

In [ ]:
df_Selected_from_List = df_Selected_from_List.drop('Fraud', axis=1)

In [ ]:
df_Selected_Scaled_Data_from_List = df_Selected_Scaled_Data_from_List.drop('Fraud', axis=1)

In [ ]:
# Feb 12th - fix assignment of weights to features
results_LIME, exec_time_LIME = generate_lime_explanations3(df_Selected_Scaled_Data_from_List, 
                                                          chosen_indices)

In [ ]:
# Unpack the results to get features_df, lime_df
features_df_LIME, lime_exp_weights_df, lime_values_structured = results_LIME

In [ ]:
#features_df_LIME

In [ ]:
#lime_exp_weights_df

In [ ]:
#lime_values_structured

### Remove Feature Names from Feature/Weights Dataframe

In [ ]:
# LIME Values needs to be limited to numeric values
xai_Lime_values_df_numeric = lime_values_structured.select_dtypes(include=[np.number])

In [ ]:
#xai_Lime_values_df_numeric

-----------------------------

### Generate Identity Metric

#### Run a Basic Test First

In [ ]:
# Select two random instances from the LIME dataframe
random_indices = np.random.choice(xai_Lime_values_df_numeric.index, size=2, replace=False)
instance_1 = xai_Lime_values_df_numeric.iloc[random_indices[0]]
instance_2 = xai_Lime_values_df_numeric.iloc[random_indices[1]]

# Compute the Euclidean distance between the selected instances
distance = get_euclidean_distance(instance_1, instance_2)
print(f"Euclidean distance between instance {random_indices[0]} and instance {random_indices[1]}: {distance:.4f}")

#### Display LIME Numeric Output

In [ ]:
#xai_Lime_values_df_numeric.head(2)

#### Retrieve Identity Score

In [ ]:
#print(features_df_LIME.index)

In [ ]:
#print(xai_Lime_values_df_numeric.index)

In [ ]:
from scipy.spatial import distance
LIME_Identity_Metric = get_identity_metric(features_df_LIME, xai_Lime_values_df_numeric, "LIME")

#### Display Identity Score Metric

In [ ]:
LIME_Identity_Number = "{:.2f}%".format(LIME_Identity_Metric)
display_text("LIME Identity Metric Score: " + LIME_Identity_Number)

In [ ]:
# Read in XAI Metric for Identity
XAI_Ident_Metric_1 = LIME_Identity_Metric

-----------------------------------

### Generate Stability Metric

#### Pre-Processing of Stability Input Data

In [ ]:
#features_df_LIME.shape

In [ ]:
#xai_Lime_values_df_numeric.index

In [ ]:
#y_test_loaded.shape

In [ ]:
#y_test_block_labels_df.shape

In [ ]:
#y_test_block_labels_df.index

In [ ]:
#print('y_test_block_labels_df')
#print(y_test_block_labels_df)

In [ ]:
# Adjust the label value input to match earlier adjustments in LIME value creations
if limit_data_block_rows > 0:
    y_test_block_labels_df = y_test_block_labels_df.iloc[:limit_data_block_rows]    

In [ ]:
#y_test_block_labels_df.shape

In [ ]:
# Assigning the column name 'Fraud'
y_test_block_labels_df.columns = ['Fraud']

In [ ]:
df = pd.DataFrame(y_test_block_labels_df)

In [ ]:
# Count the occurrence of each unique value in the 'Fraud' column
fraud_counts_label = df['Fraud'].value_counts()

# Display the counts
print("Breakdown of 'Fraud' and non-Fraud label records in df_TestData:")
print(fraud_counts_label)

In [ ]:
# Counting the occurrences of each label
label_counts = df['Fraud'].value_counts()

# Finding the label with the most entries
largest_label = label_counts.idxmax()

# Assigning it to largest_label_count
largest_label_count = label_counts[largest_label]

print("Label with most entries:", largest_label)
print("Count of this label:", largest_label_count)

#### Retrieve Stability Score

In [ ]:
# Jan 18th - use new loaded model and data - SCALED
LIME_Stability_Metric = get_stability_metric_y(xai_Lime_values_df_numeric, 
                                               y_test_block_labels_df, 
                                               largest_label, 
                                               'LIME')

#### Display Stability Score Metric

In [ ]:
LIME_Stability_Number = "{:.2f}%".format(LIME_Stability_Metric)
display_text("SHAP Stability Metric Score: " + LIME_Stability_Number)

In [ ]:
# Read in XAI Metric for Stability
XAI_Stability_Metric_2 = LIME_Stability_Metric

----------------------------

### Generate Seperability Metric

#### Retrieve Seperability Score

In [ ]:
#print(features_df_LIME.index)

In [ ]:
#print(xai_Lime_values_df_numeric.index)

In [ ]:
#features_df_LIME

In [ ]:
#xai_Lime_values_df_numeric

In [ ]:
LIME_Seperability_Metric = get_seperability_metric(features_df_LIME, 
                                                   xai_Lime_values_df_numeric, 
                                                   "LIME",
                                                   0.80, # threshold  #0.51
                                                   0.35) # tolerance)

#### Display Seperability Score Metric

In [ ]:
LIME_Seperability_Number = "{:.2f}%".format(LIME_Seperability_Metric)
display_text("LIME Seperability Metric Score: " + LIME_Seperability_Number)

In [ ]:
# Read in XAI Metric for Seperability
XAI_Seperability_Metric_3 = LIME_Seperability_Metric

-----------------------------------

### Generate Similarity Metric

#### Retrieve Similarity Score

In [ ]:
#print(features_df_LIME.index)

In [ ]:
#print(xai_Lime_values_df_numeric.index)

In [ ]:
LIME_Similarity_Metric = get_similarity_metric(features_df_LIME, 
                                               xai_Lime_values_df_numeric, 
                                               "LIME", 
                                               use_dbscan=False)

#### Display Similarity Score Metric

In [ ]:
LIME_Similarity_Number = "{:6.2f}".format(LIME_Similarity_Metric)
display_text("LIME Similarity Metric Value: " + LIME_Similarity_Number)

In [ ]:
# Read in XAI Metric for Similarity
XAI_Similarity_Metric_4 = LIME_Similarity_Metric

----------------------------

### Display Final Set of Metrics (this run)

In [ ]:
# Print the results
print(f"XAI Ident Metric 1: {XAI_Ident_Metric_1}")
print(f"XAI Stability Metric 2: {XAI_Stability_Metric_2}")
print(f"XAI Seperability Metric 1: {XAI_Seperability_Metric_3}")
print(f"XAI Similarity Metric 1: {XAI_Similarity_Metric_4}")
print(f"XAI Time Metric 5: {exec_time_LIME} seconds")

----------------------------------

## Write Out Metrics to XL

In [ ]:
write_xai_Metrics_to_XL(LIME_xai_file_path, 
                        sample, 
                        LIME_Identity_Metric, 
                        LIME_Stability_Metric, 
                        LIME_Seperability_Metric, 
                        LIME_Similarity_Metric, 
                        exec_time_LIME, 
                        df_Selected_from_List,
                        "LIME")